In [26]:
from pymongo import MongoClient
import os
import bson

def mongoCollection(connstr, db, collection):
    client = MongoClient(connstr)
    db = client[db]
    return db[collection]

import re
import datetime

collection = mongoCollection(os.environ.get('MONGODB_CONN_STR'), 'smsinfo', 'transactions')

from IPython.display import HTML, display
import tabulate

def extract_transactions(i):
    keys = ['expense_amount', 'payment_mode', 'merchant', 'datetime', 'category', 'sub_category']
#     return list(i[1]['transaction'].values()) + [str(i[1]['_id'])]
    return [i[1]['transaction'].get(key, None) for key in keys] + [str(i[1]['_id'])]

def mark_duplicate(collection, _id):
    result = collection.update_one({'_id': bson.ObjectId(_id)}, {'$set' : {'status': {'duplicate': True}}})
    print('success' if result.modified_count == 1 else 'failed')
    return result

def add_category_info(collection, _id, category='', sub_category=''):
    result = collection.update_one(
        {'_id': bson.ObjectId(_id)}, 
        {'$set' : {'transaction.category': category, 'transaction.sub_category': sub_category}})
    print('success' if result.modified_count == 1 else 'failed')
    return result

def add_category_info_for_merchant(collection, merchant, category='', sub_category=''):
    result = collection.update_many(
        {'transaction.merchant': merchant, 'transaction.category': None}, 
        {'$set' : {'transaction.category': category, 'transaction.sub_category': sub_category}})
    print(merchant + ' modified ' + str(result.modified_count))
    return result

In [32]:
year = 2020
month = 2

next_month = 1 if month == 12 else month + 1
next_year = year + 1 if month == 12 else year

monthly = list(enumerate(collection.find({'status': {'analysis_done': True},
                 'transaction.datetime' : {
                     '$gte': datetime.datetime(year,month,1), '$lt': datetime.datetime(next_year,next_month,1)}
                  }, {'transaction': 1})))

monthly_transactions = list(map(extract_transactions, monthly))

# print(monthly_transactions)

display(HTML(tabulate.tabulate(monthly_transactions, tablefmt='html')))

970,XX983,VPS*PAY Sreen.,2020-02-01 00:00:00,meet-up,alcohol,5e3a783185ce1ed04ce1b6b9
1528,XXX983,upiswiggy@icici,2020-02-01 00:00:00,food,delivery,5e3a783185ce1ed04ce1b6bc
120,XX983,VPS*DROMI .,2020-02-02 00:00:00,meet-up,food,5e3a783185ce1ed04ce1b6bd
85,XXX983,upiswiggy@icici,2020-02-02 00:00:00,food,delivery,5e3a783285ce1ed04ce1b6bf
500,CREDIT Card xx3690,PAYTM,2020-02-02 20:44:58,,,5e3a783285ce1ed04ce1b6c3
465.5,CREDIT Card xx3690,FRESHTOH3631510,2020-02-02 20:42:47,grocery,non-veg,5e3a783285ce1ed04ce1b6c4
465.5,CREDIT Card xx3690,FRESHTOH3631510 in BANGALORE,2020-02-02 20:42:48,grocery,non-veg,5e3a783285ce1ed04ce1b6c5
400,CREDIT Card xx3690,PAYTM3852398,2020-02-02 20:49:59,,,5e3a783285ce1ed04ce1b6c6
268,XX983,VPS*HORTICULT.,2020-02-03 00:00:00,grocery,vegetables,5e3a783385ce1ed04ce1b6c7
199,Debit Card xx1827,NETFLIX ENTERTAINMENT,2020-02-03 19:28:14,subscription,entertainment,5e3a783385ce1ed04ce1b6ca
500,CREDIT Card xx3690,PAYTM3852398,2020-02-03 23:14:18,,,5e3a783385ce1ed04ce1b6cb


#feb-2020 message.date > 602238600089916032

`add_category_info(collection, '5e497fb75bc90c80c1c82a81', category='purchase', sub_category='toys')`

In [29]:
merchant_category_mapping = [
    {'merchant': ['Uber', 'UBER'], 'category': 'taxi', 'sub_category': 'uber'},
    {'merchant': ['SWIGGY', 'zomato@hdfcbank', 'ZOMATO11120', 'Eat Fit', 'www.zomato.com', 'SWIGGYXL3549786', 'Paratha Corner', 'bharatpe.9040576993@icici', 'upiswiggy@icici', 'swiggyupi@axisbank'], 'category': 'food', 'sub_category': 'delivery'},
    {'merchant': ['..MATTO COFFEA_'], 'category': 'meet-up', 'sub_category': 'food'},
    {'merchant': ['bharatpe90200570491@yesbankltd', 'q22904860@ybl', 'bharatpe90200570491¡yesbankltd'], 'category': 'meet-up', 'sub_category': 'breakout'},
    {'merchant': ['NEW FRUIT LAND', 'NEW FRUITLAND'], 'category': 'grocery', 'sub_category': 'vegetables'},
    {'merchant': ['www.bigbasket.', 'M K RETAIL'], 'category': 'grocery', 'sub_category': 'all'},
    {'merchant': ['VIN*APPLE COM.', 'VSI*APPLE COM.'], 'category': 'subscription', 'sub_category': 'apps'},
    {'merchant': ['VPS*HORTICULT.'], 'category': 'grocery', 'sub_category': 'vegetables'},
    {'merchant': ['FRESHTOH3631510', 'FRESHTOH3631510 in BANGALORE', 'Freshtohome'], 'category': 'grocery', 'sub_category': 'non-veg'},
    {'merchant': ['AJIO', 'MYNTRA72883'], 'category': 'purchase', 'sub_category': 'cloths'},
    {'merchant': ['IIN*Amazon .'], 'category': 'purchase', 'sub_category': 'household'},
    {'merchant': ['amazonsellerservices.98397377@hdfcbank'], 'category': 'purchase', 'sub_category': 'toiletry'},
    {'merchant': ['Solanki medicals'], 'category': 'medical', 'sub_category': 'medicine'},
    {'merchant': ['Bharti Airtel Limited'], 'category': 'utility', 'sub_category': 'mobile'},
    {'merchant': ['Instapay BBPS'], 'category': 'utility', 'sub_category': 'bescom'},
    {'merchant': ['LIC'], 'category': 'insurance', 'sub_category': 'insurance'},
    {'merchant': ['..RAJDHANI PHOENIX_', '..ARENA_', 'bharatpe09600003315¡yesbankltd'], 'category': 'food', 'sub_category': 'eating-out'},
    {'merchant': ['The lassi club', 'paytmqr281005050101j1cog2ifqf2u@paytm'], 'category': 'food', 'sub_category': 'snacks'},
    {'merchant': ['H M LEISURE'], 'category': 'purchase', 'sub_category': 'toys'},
    {'merchant': ['PEPPERFRY64213'], 'category': 'purchase', 'sub_category': 'furniture'},
    {'merchant': ['HPCL HINDUSTAN PETROLE', 'HINDUSTAN PETROLEUM CO'], 'category': 'auto', 'sub_category': 'fuel'},
    {'merchant': ['vijualoor@okhdfcbank', 'cru5ty.d3m0nx-2@okhdfcbank'], 'category': 'others', 'sub_category': 'others'},
    {'merchant': ['BOOKMYSHOW'], 'category': 'entertainment', 'sub_category': 'movie'},
    {'merchant': ['PVR LIMITED.'], 'category': 'food', 'sub_category': 'movie'},
    {'merchant': ['PHOENIX MARKETCITY BAN'], 'category': 'auto', 'sub_category': 'parking'},
]

# bharatpe09600003315¡yesbankltd => Empire hotel

for _m in merchant_category_mapping:
    for _merchant in _m['merchant']:
        add_category_info_for_merchant(collection, _merchant, category=_m['category'], sub_category=_m['sub_category'])


Uber modified 0
UBER modified 0
SWIGGY modified 0
zomato@hdfcbank modified 0
ZOMATO11120 modified 0
Eat Fit modified 0
www.zomato.com modified 0
SWIGGYXL3549786 modified 0
Paratha Corner modified 0
bharatpe.9040576993@icici modified 0
upiswiggy@icici modified 0
swiggyupi@axisbank modified 0
..MATTO COFFEA_ modified 0
bharatpe90200570491@yesbankltd modified 0
q22904860@ybl modified 0
bharatpe90200570491¡yesbankltd modified 0
NEW FRUIT LAND modified 0
NEW FRUITLAND modified 0
www.bigbasket. modified 0
M K RETAIL modified 0
VIN*APPLE COM. modified 0
VSI*APPLE COM. modified 0
VPS*HORTICULT. modified 0
FRESHTOH3631510 modified 0
FRESHTOH3631510 in BANGALORE modified 0
Freshtohome modified 0
AJIO modified 0
MYNTRA72883 modified 0
IIN*Amazon . modified 0
amazonsellerservices.98397377@hdfcbank modified 0
Solanki medicals modified 0
Bharti Airtel Limited modified 0
Instapay BBPS modified 0
LIC modified 0
..RAJDHANI PHOENIX_ modified 0
..ARENA_ modified 0
bharatpe09600003315¡yesbankltd modified 

In [82]:
# Remove OTP

def ignore_otp(collection, gt=7602238600089916032, lte=0):
    result = collection.update_many(
        {"message.text": {'$regex': u"OTP"}},
        {'$set' : {'status.ignore': True}})
    print('otp ignored ' + str(result.modified_count))
    return result

In [83]:
gt = 0 #602238600089916032
lte = 604643260481582080
result = ignore_otp(collection, gt=gt, lte=lte)

otp ignored 303


In [103]:
def extract_documents(i):
    keys = ['expense_amount', 'payment_mode', 'merchant', 'datetime', 'category', 'sub_category']
    return [str(i[1]['_id'])] + list(i[1]['message'].values()) + [i[1].get('transaction', {}).get(key, None) for key in keys]
    
monthly = list(enumerate(collection.find({
                 "message.date": {'$gt': 602238600089916032, '$lte': 604643260481582080},
                 'status.ignore': None, "status.analysis_done": None,
                  })))

monthly_messages = list(map(extract_documents, monthly))
display(HTML(tabulate.tabulate(monthly_messages, tablefmt='html')))

5e3a783185ce1ed04ce1b6ba,Rs.1000 credited to your account. Pause your weekend plans & shop before they expire. Valid on 10% order value. Click to Claim Now! limeroad.com/u/9634ce8a,6.02262e+17,CF8E39A8-6D70-2642-0118-0C1CC1CE956D,,,,,,
5e3a783185ce1ed04ce1b6bb,"Dear Customer, SWIGGY has requested money from you on your ICICI Bank app. On approving the request, Rs 1528.00 will be debited from your account. To authorise click mobile.icicibank.com/upiPushNotifications",6.02263e+17,834C065B-8290-1E71-A6BB-4EA542C50438,,,,,,
5e3a783185ce1ed04ce1b6be,"Dear Customer, SWIGGY has requested money from you on your ICICI Bank app. On approving the request, Rs 85.00 will be debited from your account. To authorise click mobile.icicibank.com/upiPushNotifications",6.02309e+17,74E34EA6-458D-ECB4-00BB-E1F5B20CD423,,,,,,
5e3a783285ce1ed04ce1b6c2,Dear Sumit Asok amount of Rs. 500.00 has been credited to your wallet as per your request.,6.02349e+17,7AF6468D-DC96-A980-89B7-06B7EE4A084E,,,,,,
5e3a783385ce1ed04ce1b6c8,ALERT:You've withdrawn Rs.10000.00 via Debit Card xx1827 at C.V. RAMAN NAGAR on 2020-02-03:18:20:10.Avl Bal Rs.560265.31.Not you?Call 18002586161.,6.02427e+17,7176CF0A-E76F-5B83-72D4-EACC1AB3D6EB,,,,,,
5e3a783485ce1ed04ce1b6cf,"As per your request you now have a Ready Credit line of Rs 500000 ROI is 19% p.a, 1st time activation fee Rs: 500 & annual renewal fee Rs 750",6.02563e+17,A7829A17-6A1F-AF0E-5B45-6F896496E135,,,,,,
5e3e5b35dd2322b0ad5cccf8,"Txn of INR 1,294.00 done on Acct XX983 on 06-Feb-20.Info: IPS*NEW FRUIT.Avbl Bal:INR 9,281.75.Call 18002662 for dispute or SMS BLOCK 983 to 9215676766",6.02684e+17,289268D2-E071-2151-4E65-9997813355D3,,,,,,
5e3e5b35dd2322b0ad5cccfd,Great news! We've credited Rs. 10 in your Paytm First Games Bonus. Click http://m.p-y.tm/smsb to play your favourite games now & win Paytm Cash.,6.02705e+17,9F22D029-42E0-CB93-C388-5670E79FC18B,,,,,,
5e3e5b36dd2322b0ad5ccd06,Get Rs 100 back on Movies offer received for Shopping transaction. Collect here - http://amzn.in/d/8xERLXo,6.02757e+17,87967CF0-E1E6-3621-E6AC-509054C3387D,,,,,,
5e40f969637cbe765df7dd78,"You have selected a new ATM PIN for your credit card ending 0709. If you have not requested it, please contact us immediately.",6.02838e+17,E208E45D-42CA-DA13-3DDF-6BDC125F299B,,,,,,
5e40f969637cbe765df7dd7a,"Added Rs.1000.00 in your Paytm through Citibank.Your updated balance:Rs.82.25. Transaction ID: 514033854. Use Paytm for recharges, shopping and more.",6.02838e+17,3ADAC3EA-8977-7E8D-B459-6C1E732B58A4,,,,,,


In [34]:
sum_result_by_category_and_subcategory = collection.aggregate(
   [{'$match': {'status': {'analysis_done': True},
                 'transaction.datetime' : {
                     '$gte': datetime.datetime(year,month,1), '$lt': datetime.datetime(next_year,next_month,1)}
                  }},
     {
       '$group':
         {
           '_id': { 'category': "$transaction.category", 'sub_category': "$transaction.sub_category"},
           'totalAmount': { '$sum': "$transaction.expense_amount"},
           'count': { '$sum': 1 }
         }
     },
    {
        '$sort': {
            '_id': -1
        }
    }
   ]
)
display(HTML(tabulate.tabulate(list(sum_result_by_category_and_subcategory), tablefmt='html')))

"{'category': 'utility', 'sub_category': 'mobile'}",79,1
"{'category': 'utility', 'sub_category': 'internet'}",1249.56,1
"{'category': 'utility', 'sub_category': 'gas'}",869.4,1
"{'category': 'utility', 'sub_category': 'bescom'}",1350,1
"{'category': 'taxi', 'sub_category': 'uber'}",416.25,4
"{'category': 'subscription', 'sub_category': 'entertainment'}",199,1
"{'category': 'subscription', 'sub_category': 'apps'}",1616,4
"{'category': 'purchase', 'sub_category': 'toys'}",536,1
"{'category': 'purchase', 'sub_category': 'toiletry'}",1709.1,1
"{'category': 'purchase', 'sub_category': 'household'}",609,2
"{'category': 'purchase', 'sub_category': 'furniture'}",860,1


In [33]:
sum_result_by_category = collection.aggregate(
   [{'$match': {'status': {'analysis_done': True},
                 'transaction.datetime' : {
                     '$gte': datetime.datetime(year,month,1), '$lt': datetime.datetime(next_year,next_month,1)}
                  }},
     {
       '$group':
         {
           '_id': { 'category': "$transaction.category"},
           'totalAmount': { '$sum': "$transaction.expense_amount"},
           'count': { '$sum': 1 }
         }
     },
    {
        '$sort': {
            '_id': -1
        }
    }
   ]
)
display(HTML(tabulate.tabulate(list(sum_result_by_category), tablefmt='html')))

{'category': 'utility'},3547.96,4
{'category': 'taxi'},416.25,4
{'category': 'subscription'},1815,5
{'category': 'purchase'},5506.45,7
{'category': 'personal'},5817,2
{'category': 'others'},630,3
{'category': 'meet-up'},4021.75,13
{'category': 'medical'},80,1
{'category': 'insurance'},16679,1
{'category': 'grocery'},12354.6,15
{'category': 'food'},12854.8,34
